In [1]:
#!/user/bin/env python
# -*- coding:utf-8 -*-
# author: Wei Li
# creat: 2022-3-16
# modify: 2022-3-16
# function: Implement candidate elimination algorithm from Machine Learning by Thomas Mitchell (1997)
# 
# the attribute 'playtennis' indicate whether or not being suitable for palying tennis. 
# 
# For all possible day with the following attributes:
# Outlook: Sunny/Overcast/Rain
# Temporature: Hot/Mild/Cool
# Humidity: High/Normal
# Wind: False/True
# 
# 
# we use the hypothesis with the vector:
# [Outlook, Temperature, Humidity, Wind]

attributes = [['Sunny','Overcast','Rain'],
              ['Hot','Mild','Cool'],
              ['High','Normal'],
              ['False','True']]
num_attributes = len(attributes)

In [2]:
# load the examples from 'data.txt'
with open('./data.txt') as file:
    examples = []; classifications = []
    for line in file:
        dataline = line.strip('\n').split(' ')
        examples.append(tuple(dataline[1:-1]))
        classifications.append(dataline[-1])
examples = examples[1:]
classifications = classifications[1:]
examples,classifications
# examples = 
# [('Sunny', 'Hot', 'High', 'False'),
#   ('Sunny', 'Hot', 'High', 'True'),
#   ('Overcast', 'Hot', 'High', 'False'),
#   ('Rain', 'Mild', 'High', 'False'),
#   ('Rain', 'Cool', 'Normal', 'False'),
#   ('Rain', 'Cool', 'Normal', 'True'),
#   ('Overcast', 'Cool', 'Normal', 'True'),
#   ('Sunny', 'Mild', 'High', 'False'),
#   ('Sunny', 'Cool', 'Normal', 'False'),
#   ('Rain', 'Mild', 'Normal', 'False'),
#   ('Sunny', 'Mild', 'Normal', 'True'),
#   ('Overcast', 'Mild', 'High', 'True'),
#   ('Overcast', 'Hot', 'Normal', 'False'),
#   ('Rain', 'Mild', 'High', 'True')]
# classifications = 
# ['No',
#   'No',
#   'Yes',
#   'Yes',
#   'Yes',
#   'No',
#   'Yes',
#   'No',
#   'Yes',
#   'Yes',
#   'Yes',
#   'Yes',
#   'Yes',
#   'No']

([('Sunny', 'Hot', 'High', 'False'),
  ('Sunny', 'Hot', 'High', 'True'),
  ('Overcast', 'Hot', 'High', 'False'),
  ('Rain', 'Mild', 'High', 'False'),
  ('Rain', 'Cool', 'Normal', 'False'),
  ('Rain', 'Cool', 'Normal', 'True'),
  ('Overcast', 'Cool', 'Normal', 'True'),
  ('Sunny', 'Mild', 'High', 'False'),
  ('Sunny', 'Cool', 'Normal', 'False'),
  ('Rain', 'Mild', 'Normal', 'False'),
  ('Sunny', 'Mild', 'Normal', 'True'),
  ('Overcast', 'Mild', 'High', 'True'),
  ('Overcast', 'Hot', 'Normal', 'False'),
  ('Rain', 'Mild', 'High', 'True')],
 ['No',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No'])

In [3]:
# Check whether h1 is consistent with exam
def consistent(h1,exam):
    for hypt_attr, exam_attr in zip(h1,exam):
        if hypt_attr == '?' :
            continue
        elif hypt_attr == '0':
            return False
        elif hypt_attr != exam_attr:
            return False
    return True

# check whether h1 is more general than h2
def more_general(h1,h2):
    more_general_part  = []
    for h1_attr,h2_attr in zip(h1,h2):
        mg = (h1_attr == '?' or (h1_attr != '0' and h2_attr == '0') or h1_attr==h2_attr )
        more_general_part.append(mg)
    return all(more_general_part)

# update the S set when example 'd' is a positive example 
def S_update(exam,G,S):
    '''
        @param: exam, the positive example
                G, the maximum general hypothesis set G
                S, the maximum special hypothesis set S
        @return: S, the updated maximum special hypothesis set S
    '''
    old_S = list(S)
    for s in old_S:
        if s not in S:
            continue
        if not consistent(s,exam):
            S.remove(s)
            
            min_general_hs = list(s)
            for i in range(len(s)):
                if not consistent(s[i:i+1],exam[i:i+1]):
                    min_general_hs[i] = '?' if s[i] != '0' else exam[i]
                    
            S.update([h for h in min_general_hs if any([more_general(g,h) for g in G])])
            S.difference_update([h for h in S if any([more_general(h,hh) for hh in S if hh!=h])])
    return S

# update the G set when example 'd' is a negative example 
def G_update(exam,attributes,G,S):
    '''
        @param: exam, the positive example
                attributes, the value list for each attribute
                G, the maximum general hypothesis set G
                S, the maximum special hypothesis set S
        @return: G, the updated maximum general hypothesis set G
    '''
    old_G = list(G)
    for g in old_G:
        if g not in G:
            continue
        if consistent(g,exam):
            G.remove(g)
            
            min_special_hs = []
            for i in range(len(g)):
                if g[i] == '?':
                    for attr in attributes[i]:
                        if attr != exam[i]:
                            g_new = g[:i]+(attr,)+g[i+1:]
                            min_special_hs.append(g_new)
                elif g[i] != '0':
                    g_new = g[:i]+('0',)+g[i+1:]
                    min_special_hs.append(g_new)
                    
            G.update([h for h in min_special_hs if any([more_general(h,s) for s in S])])
            G.difference_update([h for h in G if any([more_general(hh,h) for hh in G if hh!=h])])
    return G

In [7]:
# the candidate_elimination algorithms implementation
def candidate_elimination(train_examples = [],classifications=[],num_attributes=0):
    '''
        @param: train_examples, the training examples
                classifications, the category for each example
                num_attributes, the number of attributes
        @return: G_H, the updated maximum general hypothesis set G_H
                 S_H, the updated maximum special hypothesis set S_H
                 
    '''
    assert(len(train_examples)!=0)
    assert(len(train_examples)==len(classifications))
    assert(len(train_examples[0])==num_attributes)
    G_H = set([('?',)*num_attributes])
    S_H = set([('0',)*num_attributes])
    print('init set for G and S: ',G_H,S_H)
    count = 1
    for exam,cla in zip(train_examples,classifications):
        if cla == 'Yes':
            G_H = {g for g in G_H if consistent(g,exam)}
            S_H = S_update(exam,G_H,S_H)
        else:
            S_H = {s for s in S_H if not consistent(s,exam)}
            G_H = G_update(exam,attributes,G_H,S_H)
        print("after example 【",count,"】:",G_H,S_H);count+=1
    return G_H,S_H

In [8]:
def main():
    hypothesis = candidate_elimination(examples,classifications,num_attributes)
    print('the final hypothesis set for G and S is:', hypothesis)

In [9]:
if __name__ == "__main__":
    main()

init set for G and S:  {('?', '?', '?', '?')} {('0', '0', '0', '0')}
after example 【 1 】: {('Overcast', '?', '?', '?'), ('?', '?', 'Normal', '?'), ('?', 'Cool', '?', '?'), ('?', 'Mild', '?', '?'), ('Rain', '?', '?', '?'), ('?', '?', '?', 'True')} {('0', '0', '0', '0')}
after example 【 2 】: {('Overcast', '?', '?', '?'), ('?', '?', 'Normal', '?'), ('?', 'Cool', '?', '?'), ('?', '?', '?', '0'), ('?', 'Mild', '?', '?'), ('Rain', '?', '?', '?')} {('0', '0', '0', '0')}
after example 【 3 】: {('Overcast', '?', '?', '?')} set()
after example 【 4 】: set() set()
after example 【 5 】: set() set()
after example 【 6 】: set() set()
after example 【 7 】: set() set()
after example 【 8 】: set() set()
after example 【 9 】: set() set()
after example 【 10 】: set() set()
after example 【 11 】: set() set()
after example 【 12 】: set() set()
after example 【 13 】: set() set()
after example 【 14 】: set() set()
the final hypothesis set for G and S is: (set(), set())
